In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from lyricsgenius import Genius
import pickle
import numpy as np

In [2]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
genius = Genius(API_KEY)
genius.remove_section_headers = True


In [3]:
df = pd.read_csv('Hot 100.csv')

In [4]:
df['chart_date'] = pd.to_datetime(df.chart_date)
df['chart_debut'] = pd.to_datetime(df.chart_debut)

In [5]:
df['consecutive_weeks'].fillna(value=0,inplace=True)
df['previous_week'].fillna(value=0,inplace=True)

In [6]:
frame = pd.DataFrame(df.dtypes)
numericals = list(frame.loc[frame.iloc[:,0]=='float64'].iloc[:,0].keys())
numericals = numericals + list(frame.loc[frame.iloc[:,0]=='int64'].iloc[:,0].keys())
for x in numericals:
    df[x] = df[x].astype('int32')

In [7]:
df.sort_values(by='chart_date',inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335195 entries, 100546 to 79691
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   chart_position     335195 non-null  int32         
 1   chart_date         335195 non-null  datetime64[ns]
 2   song               335195 non-null  object        
 3   performer          335195 non-null  object        
 4   song_id            335195 non-null  object        
 5   instance           335195 non-null  int32         
 6   time_on_chart      335195 non-null  int32         
 7   consecutive_weeks  335195 non-null  int32         
 8   previous_week      335195 non-null  int32         
 9   peak_position      335195 non-null  int32         
 10  worst_position     335195 non-null  int32         
 11  chart_debut        335195 non-null  datetime64[ns]
 12  chart_url          335195 non-null  object        
dtypes: datetime64[ns](2), int32(7), object(4

In [9]:

unique_df = df.drop_duplicates(subset=['song_id'])

In [10]:
def get_lyrics_genius(row):
    ans = float('nan')
    title = row['song']
    artist = row['performer']
    try:
        song = genius.search_song(title, artist=artist)
    except:
        ans = float('nan')
    try:
        song.lyrics
    except:
        ans = float('nan')
    else:
        if song.lyrics[:len(title)]==title:
            ans = song.lyrics[len(title)+len(' lyrics'):]
        else:
            ans = float('nan')
    return(ans)


In [23]:
start = 4500
stop = start+1500
subset_df = unique_df.iloc[start:stop]

In [24]:
subset_df['lyrics'] = subset_df.apply(get_lyrics_genius,axis=1)

Searching for "Keep On Trying" by Bobby Vee...
Done.
Searching for "It Ain't No Big Thing" by Radiants...
Done.
Searching for "Peanuts (La Cacahuata)" by The Sunglows...
No results found for: 'Peanuts (La Cacahuata) The Sunglows'
Searching for "You Can Have Her" by The Righteous Brothers...
Done.
Searching for "A Walk In The Black Forest" by Horst Jankowski...
Specified song does not contain lyrics. Rejecting.
Searching for "The Climb" by The Kingsmen...
Done.
Searching for "Love Is A 5-Letter Word" by James Phelps...
No results found for: 'Love Is A 5-Letter Word James Phelps'
Searching for "Lipstick Traces (On A Cigarette)" by The O'Jays...
Done.
Searching for "Tommy" by Reparata And The Delrons...
Done.
Searching for "Engine Engine #9" by Roger Miller...
Done.
Searching for "L-O-N-E-L-Y" by Bobby Vinton...
Done.
Searching for "Shakin' All Over" by The Guess Who...
Done.
Searching for "And I Love Him" by Esther Phillips...
Done.
Searching for "Last Chance To Turn Around" by Gene Pitn

C:\Users\kapta\AppData\Local\Temp\ipykernel_488\3462107902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['lyrics'] = subset_df.apply(get_lyrics_genius,axis=1)


In [25]:
subset_df.to_pickle('./dataframe'+str(start)+'-'+str(stop)+'.pkl')